<a href="https://colab.research.google.com/github/ichekhovskikh/fasttext/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymorphy2[fast]

import gensim
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk
from gensim.models.fasttext import FastText

nltk.download('stopwords')

from nltk.corpus import stopwords
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 70 текстов.

И мы проверим нашу модель на глаз, используя тестовый корпус, который содержит 7 документов.

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать)

In [0]:
#@title Введите путь к файлам исходной базы статей:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/data.json' #@param {type: "string"}
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/test.json' #@param {type: "string"}

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

Лемматизация каждого слова статьи:

In [0]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

Удаление стоп слов:

In [0]:
russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

Предобработка текста статьи:

In [0]:
def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = remove_stopwords(normalized_text)
    normalized_text = list(lemmatize(normalized_text))
    return normalized_text

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for article in corpus:
            normalized_text = advanced_preprocess(article['text'])
            yield normalized_text

Получение исходного текста статьи по индексу:

In [0]:
def get_article_text_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

Загружаем корпуса:

In [0]:
train_corpus = list(read_corpus(train_path))
test_corpus = list(read_corpus(test_path))

Давайте посмотрим на учебный и тестовый корпуса:

In [0]:
print(train_corpus[:2])

[['общественный', 'жизнь', 'экономика', 'рынок', 'каждый', 'готовить', 'блин', 'сковорода', 'антипригарный', 'покра', 'тие', 'знать', 'важный', 'температура', 'приготовление', 'который', 'зависеть', 'цвета', 'форма', 'изделие', 'домаш', 'условие', 'некоторый', 'колебание', 'цвет', 'толщина', 'форма', 'вполне', 'приемлемый', 'промышленный', 'произа', 'водство', 'блин', 'сложный', 'процесс', 'неоднородность', 'готовый', 'продукт', 'становиться', 'больший', 'пра', 'пятствие', 'успешный', 'реали', 'зации', 'торговый', 'сеть', 'новое', 'технический', 'решение', 'стабилизация', 'качество', 'промышленный', 'ный', 'производство', 'блин', 'разрабо', 'таль', 'специалист', 'шведский', 'фирма', 'formcook', 'промышленный', 'печь', 'термообработка', 'самый', 'разно', 'образный', 'продукт', 'мясо', 'птица', 'рыба', 'овощ', 'пять', 'год', 'поставлять', 'россия', 'компания', 'агро', 'надёжный', 'высокопроиза', 'водительный', 'оборудование', 'успеть', 'завоевать', 'авторитет', 'российский', 'специалист'

In [0]:
print(test_corpus[:2])

[['язык', 'ассемблер', 'машинный', 'ориентировать', 'язык', 'низкий', 'уровень', 'команда', 'прямо', 'соответствовать', 'отдельный', 'команда', 'машина', 'также', 'предоставлять', 'дополнительный', 'возможность', 'облегчение', 'такой', 'макрокоманда', 'выражение', 'средство', 'обеспечение', 'модульность', 'программа', 'рассматриваться', 'автокод', 'расширить', 'конструкция', 'язык', 'высокий', 'уровень', 'являться', 'существенно', 'платформо', 'зависимый', 'язык', 'ассемблер', 'различный', 'аппаратный', 'платформа', 'несовместимый', 'хотя', 'мочь', 'целое', 'подобный', 'русский', 'язык', 'именоваться', 'просто', 'ассемблер', 'типичный', 'выражение', 'тип', 'писать', 'программа', 'ассемблер', 'строго', 'говорить', 'неверный', 'ассемблер', 'именоваться', 'утилит', 'трансляция', 'программа', 'язык', 'ассемблер', 'объектный', 'код', 'компьютер'], ['перитонит', 'развиваться', 'вследствие', 'бактериальный', 'инфицирование', 'брюшной', 'полость', 'подавлять', 'большинство', 'пациент', 'перито

# Обучение модели
## Создание объекта FastText
Теперь мы создадим модель FastText с векторным размером 80 слов и перебираем учебный корпус 100 раз.
Параметры обучения:
- size: размерность вектора
- window: «окно», в котором рассматривается контекст
- min_count: минимальная частота появления слова, если частота ниже заданной – игнорировать
- sample: максимальная частота появления слова, если частота выше заданной – игнорировать
- sg: cbow (0) или skip-gram (1)
- epochs: количество иттераций

In [0]:
#@title Укажите параметры обучения модели:
vector_size = 80 #@param
window =  5 #@param
epochs =  100 #@param
min_count = 2 #@param
alpha = 0.001 #@param
learning_method = "skip-gram" #@param ["skip-gram", "CBOW"] {type:"raw"}

sg = 1
if (learning_method == "CBOW"):
    sg = 0

In [0]:
model = FastText(
    size=vector_size, 
    window=window,
    min_count=min_count,
    alpha=alpha,
    sg=sg)
model.build_vocab(sentences=train_corpus)
%time model.train(sentences=train_corpus, total_examples=model.corpus_count, epochs=epochs)

CPU times: user 37.4 s, sys: 37.8 ms, total: 37.4 s
Wall time: 20.5 s


# Опредлим функции для преобразования вектора слов в вектор текста

Обратная сглаженная частота (вес слова), где
- a: гиперпараметр, который обычно равен 0.001 (конфигурация, внешняя по отношению к модели, значение которой невозможно оценить по данным):

In [0]:
a = 0.001

def sif(word, sentence):
    frequency = sentence.count(word) / len(sentence)
    return a / (a + frequency) 
    

Получить вектор текста:

In [0]:
def sentence_vector(model, sentence):
    result_vector = []
    for word in sentence:
        weight = sif(word, sentence)
        try:
            word_vector = model.wv[word]
            weighted_word_vector = [weight * elem for elem in word_vector]
            if not result_vector:
                result_vector = weighted_word_vector
            else: 
                result_vector = [x + y for x, y in zip(result_vector, weighted_word_vector)]
        except:
            continue
    return result_vector

# Поиск похожих текстов на основе обученной модели

Определение косинусной близости двух векторов:

In [0]:
def cosine_similarity(first_vector, second_vector):
    numerator = sum([x * y for x, y in zip(first_vector, second_vector)])
    first_norm = pow(sum([x * x for x in first_vector]), 0.5)
    second_norm = pow(sum([x * x for x in second_vector]), 0.5)
    return numerator / (first_norm * second_norm)

Поиск похожих текстов из корпуса

In [0]:
def sims(model, corpus, corpus_path, sentence, count = 4):
    result = []
    compared_vector = sentence_vector(model, sentence)
    for index, text in enumerate(corpus):
        text_vector = sentence_vector(model, text)
        similarity = cosine_similarity(compared_vector, text_vector)
        result.append([similarity, get_article_text_by_index(index, corpus_path)])
    return reversed(sorted(result,  key=lambda x: x[0]))[:count]

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [0]:
doc_id = random.randint(0, len(train_corpus) - 1)
doc = train_corpus[doc_id]
sims = sims(model, train_corpus, train_path, doc)
print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text_by_index(doc_id, train_path)))
num = 0
for similarity, text in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, text))



ТЕКСТ ИСХДНОГО ДОКУМЕНТА (318): « РОССИЙСКАЯ ФЕДЕРАЦИЯ ФЕДЕРАЛЬНАЯ СЛУЖБА ПО ИНТЕЛЛЕКТУАЛЬНОЙ СОБСТВЕННОСТИ ПАТЕНТАМ И ТОВАРНЫМ ЗНАКАМ 51 МПК G06F 17 30 2006 01 19 RU 11 2 390 833 13 C2 12 ОПИСАНИЕ ИЗОБРЕТЕНИЯ К ПАТЕНТУ 21 22 Заявка 2005113002 09 28 04 2005 24 Дата начала отсчета срока действия патента 28 04 2005 30 Конвенционный приоритет 29 04 2004 US 10 834 483 43 Дата публикации заявки 10 11 2006 45 Опубликовано 27 05 2010 Бюл 15 56 Список документов цитированных в отчете о поиске Shen H T et al Finding semantically related images in the WWW Proceedings of the eighth ACM international conference on Multimedia Marina del Rey California United States 2000 RU 2177174 C1 20 12 2001 EP 0964340 A 15 12 1999 US 6112202 A 29 08 2000 US 6665837 B1 16 12 2003 US 5546517 A 13 08 1996 Адрес для переписки 129090 Москва ул Б Спасская 25 стр 3 ООО Юридическая фирма Городисский и Партнеры пат пов Ю Д Кузнецову рег 595 72 Автор ы ЦАЙ Дэн US ВЭНЬ Цзи Жун US МА Вэй Ин US ХЭ Сяофэй US 73 Патентооблада

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [0]:
doc_id = random.randint(0, len(test_corpus) - 1)
doc = test_corpus[doc_id]
sims = sims(model, train_corpus, train_path, doc)
print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text_by_index(doc_id, test_path)))
num = 0
for similarity, text in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, text))

Тестирование на малых данных для выявления ошибок в ходе написания кода

In [0]:
from gensim.test.utils import common_texts

print(common_texts)

model = FastText(size=80, window=4, min_count=2, sample=1e-2, sg=1)
model.build_vocab(sentences=common_texts)
model.train(sentences=common_texts, total_examples=model.corpus_count, epochs=10)

first_vector = sentence_vector(model, ['human', 'interface', 'computer'])
second_vector = sentence_vector(model, ['human', 'user', 'interface', 'system'])
print(cosine_similarity(first_vector, second_vector))

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]
0.6390201313631765


# Поиск похожих научных документов
Выполните поиск похожих научных статей

In [0]:
#@title Укажите путь к тексту статьи в формате *.txt или введите текст статьи:
article_text = '' #@param {type: "string"}
article_path = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/example1.txt' #@param {type: "string"}

if (article_text == ''):
    with urllib.request.urlopen(article_path) as article_url:
      article_text = article_url.read()


In [0]:
normalized_text = advanced_preprocess(article_text)
sims = sims(model, train_corpus, train_path, normalized_text)

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА «{}»\n'.format(article_text)
num = 0
for similarity, text in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, text))